# Bank Deposit Enrollment Classification

In this notebook, we'll illustrate classification focusing on introducing random forests and boosted trees. We will also look at an "economically" motivated evaluation measure.

The data are from S. Moro, P. Cortez and P. Rita (2014) “A Data-Driven Approach to Predict the Success of Bank Telemarketing. Decision Support Systems,” Decision Support Systems 62, 22-31.

The outcome variable is a binary variable indicating whether a person subscribes to a bank term deposit (y = 1).

# Python Setup

As usual, we'll start by importing libraries we're going to make use of.

In [1]:
# Partial dependence plots with binary data result in an error in sklearn 1.6.1
# Make sure using an earlier version of sklearn until bug is fixed
!pip install scikit-learn==1.5.2

import sklearn
print(sklearn.__version__)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 44.2 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
1.5.2


In [ ]:
# Import relevant packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.inspection import PartialDependenceDisplay


!pip install shap
import shap


# Load and examine data

We'll import the data from the course github repository.

In [ ]:
# Import data
file = "https://raw.githubusercontent.com/chansen776/MBA-ML-Course-Materials/main/Data/bank-additional-full.csv"
rawdata = pd.read_csv(file, sep = ";")
print(rawdata.shape)
print(rawdata.columns)
print(rawdata.dtypes)

We're going to recode the outcome variable from a string ('yes','no') to a binary (1,0).

In [ ]:
# Recode outcome from "yes" and "no" to 1 and 0
rawdata['y'] = rawdata['y'].replace({'no': 0, 'yes': 1})

Note that `duration` is essentially an outcome. You don't know this before the contact and `duration = 0` implies `y = "no"`. We're going to drop this variable because we do want to use it as a predictor.

**What do you think would happen if we included it?**

In [ ]:
# Drop duration column
rawdata = rawdata.drop(columns = ['duration'])



`pdays = 999` means the person was never contacted before. Otherwise, it is the number of days since the last contact. Let's redefine variables so that they numerically make sense.



In [ ]:
# Create variable indicating not previously contacted and replace 999's in pdays with 0's
rawdata['never_contacted'] = np.where(rawdata['pdays'] == 999, 1, 0)
rawdata['never_contacted'] = rawdata['never_contacted'].astype('category')
rawdata['pdays'] = np.where(rawdata['pdays'] == 999, 0, rawdata['pdays'])
rawdata['pdays'].describe()


Finally, we set aside a hold out data set for validation purposes.

In [ ]:
# Split the data into training (80%) and validation (20%) sets
train, val = train_test_split(rawdata, test_size = 0.2, random_state = 94)

# Logistic Regression

Let's start out by building a logistic regression model to use for predicting customer outcomes.

In [ ]:
# Set data up for use with logistic regression
X_train = train.drop(columns = ['y'])
y_train = train['y']
X_val = val.drop(columns = ['y'])
y_val = val['y']

# Get dummy variables for our categorical features. Because we are not doing regularization or any kind of variable selection,
# we will drop one of each set of dummies.
X_train = pd.get_dummies(X_train, drop_first = True)
X_val = pd.get_dummies(X_val, drop_first = True)

# Align validation data to ensure same columns as training data
X_val = X_val.reindex(columns=X_train.columns, fill_value=0)

# Fitting the logistic regression
logistic_model = LogisticRegression(max_iter=1000, random_state=94, penalty = None)
logistic_model.fit(X_train, y_train)

# Make predictions on the validation set
y_pred_logistic = logistic_model.predict(X_val)
y_pred_prob_logistic = logistic_model.predict_proba(X_val)[:, 1]

# Evaluate the model
logistic_classification_metrics = classification_report(y_val, y_pred_logistic, output_dict=True)
print(pd.DataFrame(logistic_classification_metrics))

ConfusionMatrixDisplay.from_predictions(y_val, y_pred_logistic)
plt.show()

# ROC Curve
fpr, tpr, thresholds = roc_curve(y_val, y_pred_prob_logistic)
roc_auc_logistic = roc_auc_score(y_val, y_pred_prob_logistic)

plt.plot(fpr, tpr, label=f'Logistic Regression (area = {roc_auc_logistic:.2f})')
plt.plot([0, 1], [0, 1], color='black', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.show()


We get almost 90% accuracy!

**How excited should we be?**

# Constant Model (Ignore Covariates)

Let's fit a model with no predictors at all. That is we are going to predict the same value for literally everyone in the sample. The predicted probability is just going to be the fraction of individuals in the sample that actually subscribe to a deposit.

In [ ]:
# Constant model
y_pred_constant = np.zeros(len(y_val))
y_pred_prob_constant = np.zeros(len(y_val))+np.mean(y_train)

# Evaluate the model
constant_classification_metrics = classification_report(y_val, y_pred_constant, output_dict=True)
print(pd.DataFrame(constant_classification_metrics))

ConfusionMatrixDisplay.from_predictions(y_val, y_pred_constant)
plt.show()

# ROC Curve
fpr_con, tpr_con, thresholds_con = roc_curve(y_val, y_pred_prob_constant)
roc_auc_constant = roc_auc_score(y_val, y_pred_prob_constant)

plt.plot(fpr, tpr, label=f'Logistic Regression (area = {roc_auc_logistic:.2f})')
plt.plot(fpr_con, tpr_con, label=f'Constant (area = {roc_auc_constant:.2f})')
plt.plot([0, 1], [0, 1], color='black', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.show()

We get almost the same accuracy as the logistic regression (just under 90%).

**Should we be excited?**

**What do we care about?**

# Random Forest

Now let's build a random forest model.

A random forest is a prediction rule obtained by averaging many tree prediction rules where each tree prediction rule is trained using a random (sub-)set of the training data. (Jargon: Repeatedly drawing samples from the original data is called a *bootstrap*. Random forests are an example of *bagging* = bootstrap aggregation. L. Breiman (1996) “Bagging predictors” Machine Learning 26, 123-140.)

The basic random forest algorithm in *pseudo-code*:



```
for b = 1,...,B
  1. Generate data y_b,X_b by resampling from the training data
  2. Use data y_b, X_b to estimate tree prediction rule tree_b(x)
end

Random forest prediction rule = rf(x) = (1/B)sum_b tree_b(x)
```

To estimate the random forest, need to choose number of trees (*B*), any tree tuning parameters, and how many variables to include in each bootstrap sample. Intuition for random forests says we should build many low-bias trees for averaging:

*   Choose large *B*
*   Make sure trees are "deep"

Defaults seem to work pretty well.

However, here we are going to choose tuning parameters by cross-validation.

Tuning parameters were chosen using 5 fold cross-validation by running the following code:

```
#######################################################
# Takes approximately 30 minutes to run online
# Best tuning parameters: n_estimators = 1000 , min_samples_leaf = 15

# Random forest classifier. Choosing tuning parameters by cross-validation

# Set data up for use with random forest
X_train = train.drop(columns = ['y'])
y_train = train['y']
X_val = val.drop(columns = ['y'])
y_val = val['y']

# Get dummy variables for our categorical features.
# We are just going to keep all the dummies
X_train = pd.get_dummies(X_train, drop_first = False)
X_val = pd.get_dummies(X_val, drop_first = False)

# Align validation data to ensure same columns as training data
X_val = X_val.reindex(columns=X_train.columns, fill_value=0)

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 250, 500, 1000],
    'min_samples_leaf': [1, 15, 30, 60, 120]
}

# Initialize the RandomForestClassifier
rf = RandomForestClassifier(random_state=94)

# Initialize GridSearchCV with 5-fold cross-validation
cv = KFold(n_splits=5, shuffle=True, random_state=94)
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=cv, scoring='neg_log_loss', n_jobs=-1)

# Fit the grid search model
grid_search.fit(X_train, y_train)

# Print the best parameters
print(f"Best parameters: {grid_search.best_params_}")

# Evaluate the best model on the test data
best_rf = grid_search.best_estimator_
y_pred_rf = best_rf.predict(X_val)
y_pred_prob_rf = best_rf.predict_proba(X_val)[:, 1]

# Evaluate the model
rf_classification_metrics = classification_report(y_val, y_pred_rf, output_dict=True)
print(pd.DataFrame(rf_classification_metrics))

ConfusionMatrixDisplay.from_predictions(y_val, y_pred_rf)
plt.show()

# ROC Curve
fpr_rf, tpr_rf, thresholds_rf = roc_curve(y_val, y_pred_prob_rf)
roc_auc_rf = roc_auc_score(y_val, y_pred_prob_rf)

plt.plot(fpr, tpr, label=f'Logistic Regression (area = {roc_auc_logistic:.2f})')
plt.plot(fpr_rf, tpr_rf, label=f'Random Forest (area = {roc_auc_rf:.2f})')
plt.plot([0, 1], [0, 1], color='black', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.show()


```



To make sure things were stabilized, OOB error across trees was checked with the following code:



```
#######################################################
# Takes approximately 5 minutes to run online

# Initialize lists to store OOB scores and number of trees
n_estimators_range = range(1, 1001, 20)  # Number of trees from 1 to 1000
oob_scores = []

rf_oob = RandomForestClassifier(warm_start=True,
        min_samples_leaf=15,
        random_state=94,
        oob_score=True,  # Enable OOB score
        bootstrap=True   # Ensure bootstrap sampling is used
    )

# Loop through the number of estimators and compute OOB scores
for n_estimators in n_estimators_range:
    rf_oob.set_params(n_estimators=n_estimators)
    rf_oob.fit(X_train, y_train)  # Fit the model
    oob_scores.append(rf_oob.oob_score_)  # Collect the OOB score

# Plot OOB performance
plt.figure(figsize=(10, 6))
plt.plot(n_estimators_range, oob_scores, label="OOB Score")
plt.xlabel("Number of Trees")
plt.ylabel("Out-of-Bag (OOB) Score")
plt.title("OOB Performance vs. Number of Trees in Random Forest")
plt.grid()
plt.legend()
plt.show()
```



In [ ]:
# Set data up for use with random forest
X_train = train.drop(columns = ['y'])
y_train = train['y']
X_val = val.drop(columns = ['y'])
y_val = val['y']

# Get dummy variables for our categorical features.
# We are just going to keep all the dummies
X_train = pd.get_dummies(X_train, drop_first = False)
X_val = pd.get_dummies(X_val, drop_first = False)

# Align validation data to ensure same columns as training data
X_val = X_val.reindex(columns=X_train.columns, fill_value=0)

# Random forest classifier
rf_model = RandomForestClassifier(n_estimators=1000, min_samples_leaf=15, random_state=94)
rf_model.fit(X_train, y_train)

# Make predictions on the validation set
y_pred_rf = rf_model.predict(X_val)
y_pred_prob_rf = rf_model.predict_proba(X_val)[:, 1]

# Evaluate the model
rf_classification_metrics = classification_report(y_val, y_pred_rf, output_dict=True)
print(pd.DataFrame(rf_classification_metrics))

ConfusionMatrixDisplay.from_predictions(y_val, y_pred_rf)
plt.show()

# ROC Curve
fpr_rf, tpr_rf, thresholds_rf = roc_curve(y_val, y_pred_prob_rf)
roc_auc_rf = roc_auc_score(y_val, y_pred_prob_rf)

plt.plot(fpr, tpr, label=f'Logistic Regression (area = {roc_auc_logistic:.2f})')
plt.plot(fpr_rf, tpr_rf, label=f'Random Forest (area = {roc_auc_rf:.2f})')
plt.plot([0, 1], [0, 1], color='black', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.show()

Random forest is mildly more accurate than logistic regression - with a slightly higher AUC.

**Is accuracy what we care about?**

# Boosted Trees

Now let's build a boosted tree.

A boosted tree is a prediction rule obtained by adding together many tree prediction rules where each tree prediction rule is trained to fit the prediction errors from some previous model. (Boosted trees are an example of *boosting*. R. E. Schapire. The strength of weak learnability. Machine Learning, 5:197–227, 1990.)

The basic boosted tree algorithm in *pseudo-code*:



```
Initialize residuals as r_i = y_i
for b = 1,...,B
  1. Fit tree based prediction rule to outcomes r_i to obtain prediction rule f_b(x)
  2. Update residuals to r_i = r_i - lambda f_b(x_i)
end

Boosted tree prediction rule = boost(x) = sum_b lambda f_b(x)
```

To estimate the boosted tree model, need to choose number of trees (*B*), any tree tuning parameters, and learning rate lambda. Intuition for boosting says we should improve fit at a slow rate:

*   Choose lambda small ($\ll$1)
*   Make sure trees are "shallow"

Here we are going to choose tuning parameters (B,lambda,tree depth) by cross-validation.

Tuning parameters were chosen using 5 fold cross-validation by running the following code:



```
#######################################################
# Takes approximately 45 minutes to run online
# Best tuning parameters: 'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 60

# Gradient Boosting Classifier. Choosing tuning parameters by cross-validation

# Set data up for use with GradientBoostingClassifier
X_train = train.drop(columns=['y'])
y_train = train['y']
X_val = val.drop(columns=['y'])
y_val = val['y']

# Get dummy variables for our categorical features.
X_train = pd.get_dummies(X_train, drop_first=False)
X_val = pd.get_dummies(X_val, drop_first=False)

# Align validation data to ensure same columns as training data
X_val = X_val.reindex(columns=X_train.columns, fill_value=0)

# Define the parameter grid
param_grid = {
    'learning_rate': [0.01, 0.1, 1],
    'max_depth': [2, 3, 4, 5, 6],
    'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
}

# Initialize the GradientBoostingClassifier
gbc = GradientBoostingClassifier(random_state=94)

# Initialize GridSearchCV with 5-fold cross-validation
cv = KFold(n_splits=5, shuffle=True, random_state=94)
grid_search = GridSearchCV(estimator=gbc, param_grid=param_grid, cv=cv, scoring='neg_log_loss', n_jobs=-1)

# Fit the grid search model
grid_search.fit(X_train, y_train)

# Print the best parameters
print(f"Best parameters: {grid_search.best_params_}")

# Evaluate the best model on the test data
best_gbc = grid_search.best_estimator_
y_pred_gbc = best_gbc.predict(X_val)
y_pred_prob_gbc = best_gbc.predict_proba(X_val)[:, 1]

# Evaluate the model
gbc_classification_metrics = classification_report(y_val, y_pred_gbc, output_dict=True)
print(pd.DataFrame(gbc_classification_metrics))

ConfusionMatrixDisplay.from_predictions(y_val, y_pred_gbc)
plt.show()

# ROC Curve
fpr_gbc, tpr_gbc, thresholds_gbc = roc_curve(y_val, y_pred_prob_gbc)
roc_auc_gbc = roc_auc_score(y_val, y_pred_prob_gbc)

plt.plot(fpr, tpr, label=f'Logistic Regression (area = {roc_auc_logistic:.2f})')
plt.plot(fpr_rf, tpr_rf, label=f'Random Forest (area = {roc_auc_rf:.2f})')
plt.plot(fpr_gbc, tpr_gbc, label=f'Gradient Boosting (area = {roc_auc_gbc:.2f})')
plt.plot([0, 1], [0, 1], color='black', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.show()
```



In [ ]:
# Set data up for use with GradientBoostingClassifier
X_train = train.drop(columns=['y'])
y_train = train['y']
X_val = val.drop(columns=['y'])
y_val = val['y']

# Get dummy variables for our categorical features.
X_train = pd.get_dummies(X_train, drop_first=False)
X_val = pd.get_dummies(X_val, drop_first=False)

# Align validation data to ensure same columns as training data
X_val = X_val.reindex(columns=X_train.columns, fill_value=0)

# Define the parameter grid
param_grid = {
    'learning_rate': [0.01, 0.1, 1],
    'max_depth': [2, 3, 4, 5, 6],
    'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
}

# Initialize the GradientBoostingClassifier
gbc_model = GradientBoostingClassifier(random_state=94, learning_rate=0.1, max_depth=4, n_estimators=60)
gbc_model.fit(X_train, y_train)

# Evaluate the model on the test data
y_pred_gbc = gbc_model.predict(X_val)
y_pred_prob_gbc = gbc_model.predict_proba(X_val)[:, 1]

# Evaluate the model
gbc_classification_metrics = classification_report(y_val, y_pred_gbc, output_dict=True)
print(pd.DataFrame(gbc_classification_metrics))

ConfusionMatrixDisplay.from_predictions(y_val, y_pred_gbc)
plt.show()

# ROC Curve
fpr_gbc, tpr_gbc, thresholds_gbc = roc_curve(y_val, y_pred_prob_gbc)
roc_auc_gbc = roc_auc_score(y_val, y_pred_prob_gbc)

plt.plot(fpr, tpr, label=f'Logistic Regression (area = {roc_auc_logistic:.2f})')
plt.plot(fpr_rf, tpr_rf, label=f'Random Forest (area = {roc_auc_rf:.2f})')
plt.plot(fpr_gbc, tpr_gbc, label=f'Gradient Boosting (area = {roc_auc_gbc:.2f})')
plt.plot([0, 1], [0, 1], color='black', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.show()

Performs about the same as the random forest on this example with these tuning choices.

With boosted trees, we know that after some point, increasing *B* is not going to help and will eventually start to hurt as we overspecialize to the training data (i.e. as we overfit).

Rather than use cross-validation to choose tuning parameters (especially *B*), we can use **early stopping** instead. Early stopping monitors prediction performance as we increase *B* and stops as soon as predictive performance remains level/starts to deteriorate.

Early stopping can save computation time and simultaneously perform well out-of-sample. It is available with procedures that adapt the model in a step-by-step fashion (which is most of things people actually do).

In [ ]:
# Set data up for use with GradientBoostingClassifier
X_train = train.drop(columns=['y'])
y_train = train['y']
X_val = val.drop(columns=['y'])
y_val = val['y']

# Get dummy variables for our categorical features.
X_train = pd.get_dummies(X_train, drop_first=False)
X_val = pd.get_dummies(X_val, drop_first=False)

# Align validation data to ensure same columns as training data
X_val = X_val.reindex(columns=X_train.columns, fill_value=0)

# Define the parameter grid
param_grid = {
    'learning_rate': [0.01, 0.1, 1],
    'max_depth': [2, 3, 4, 5, 6]
}

# Early stopping criteria
best_model = None
best_score = float('inf')

for learning_rate in param_grid['learning_rate']:
    for max_depth in param_grid['max_depth']:
        # Initialize the GradientBoostingClassifier with early stopping
        gbcES = GradientBoostingClassifier(
            learning_rate=learning_rate,
            max_depth=max_depth,
            n_estimators=200,
            validation_fraction=0.2,
            n_iter_no_change=10,
            tol=1e-4,
            random_state=94
        )

        # Fit the model
        gbcES.fit(X_train, y_train)

        # Evaluate the validation loss from the internal validation split
        val_loss = min(gbcES.train_score_[-10:])  # Use the lowest validation loss from early stopping

        if val_loss < best_score:
            best_score = val_loss
            best_model = gbcES

# Print the best parameters
print(f"Best model parameters: learning_rate={best_model.learning_rate}, max_depth={best_model.max_depth}")

# Evaluate the best model on the test data
y_pred_gbcES = best_model.predict(X_val)
y_pred_prob_gbcES = best_model.predict_proba(X_val)[:, 1]

# Evaluate the model
gbcES_classification_metrics = classification_report(y_val, y_pred_gbcES, output_dict=True)
print(pd.DataFrame(gbcES_classification_metrics))

ConfusionMatrixDisplay.from_predictions(y_val, y_pred_gbcES)
plt.show()

# ROC Curve
fpr_gbcES, tpr_gbcES, thresholds_gbcES = roc_curve(y_val, y_pred_prob_gbcES)
roc_auc_gbcES = roc_auc_score(y_val, y_pred_prob_gbcES)

plt.plot(fpr, tpr, label=f'Logistic Regression (area = {roc_auc_logistic:.2f})')
plt.plot(fpr_rf, tpr_rf, label=f'Random Forest (area = {roc_auc_rf:.2f})')
plt.plot(fpr_gbc, tpr_gbc, label=f'Gradient Boosting (area = {roc_auc_gbc:.2f})')
plt.plot(fpr_gbcES, tpr_gbcES, label=f'Gradient Boosting Early (area = {roc_auc_gbcES:.2f})')
plt.plot([0, 1], [0, 1], color='black', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.show()

Performance using early stopping is essentially identical to that obtained by full cross-validation in this example.

In [ ]:
# Retrain using the best parameters without early stopping
long_model = GradientBoostingClassifier(
    learning_rate=best_model.learning_rate,
    max_depth=best_model.max_depth,
    n_estimators=200,
    random_state=94
)
long_model.fit(X_train, y_train)


train_acc_without = []
val_acc_without = []

train_acc_with = []
val_acc_with = []

for i, (train_pred, val_pred) in enumerate(
    zip(
        long_model.staged_predict(X_train),
        long_model.staged_predict(X_val),
    )
):
    train_acc_without.append(accuracy_score(y_train, train_pred))
    val_acc_without.append(accuracy_score(y_val, val_pred))

for i, (train_pred, val_pred) in enumerate(
    zip(
        best_model.staged_predict(X_train),
        best_model.staged_predict(X_val),
    )
):
    train_acc_with.append(accuracy_score(y_train, train_pred))
    val_acc_with.append(accuracy_score(y_val, val_pred))



fig, axes = plt.subplots(ncols=2, figsize=(12, 4))

axes[0].plot(train_acc_without, label="gbm_full")
axes[0].plot(train_acc_with, label="gbm_early_stopping")
axes[0].set_xlabel("Boosting Iterations")
axes[0].set_ylabel("Accuracy (Training)")
axes[0].legend()
axes[0].set_title("Training Accuracy")

axes[1].plot(val_acc_without, label="gbm_full")
axes[1].plot(val_acc_with, label="gbm_early_stopping")
axes[1].set_xlabel("Boosting Iterations")
axes[1].set_ylabel("Accuracy (Validation)")
axes[1].legend()
axes[1].set_title("Validation Accuracy")

Early stopping also saves a lot of computation!

# Evaluate using "expected profit"

We might want to use an "economic" criterion to evaluate our models.

If we were thinking about how to value a contact resulting in a new account, we might want to consider



1. Initial Deposit and Balance Maintenance over time
2. Interest and Fee Income
3. Cross-Selling Opportunities
4. Customer Retention and Longevity
5. Cost of Acquisition and Servicing
6. Discount Rate

We're not going to consider most of these in our little example.

For simplicity, let's suppose that there is a cost $C = 100$ of contacting an individual.

Let's suppose that a customer who is contacted and opens an account has the following characteristics:



*   Initial deposit = 1000, average maintained balance = 2000
*   Assessed fees = 50/year
*   Takes out credit card that generates 100 in fees and interest/year
*   Stays with the bank for 5 years

Let's further assume that the bank



*   gets 5%/year in interest from loans made from deposits
*   incurs costs of 100 initially and 20/year for account maintenance, setup, ...
*   has a discount rate of 5%

From this, we have that the net revenue of the customer is (100+50+100-100-20 = 130) in year 1 and (100+50+100-20 = 230) in years 2-5. Lifetime value of the customer is then approximately $R = \frac{130}{1.05} + \sum_{j=2}^{5}\frac{230}{1.05^j} = 904 \approx 900.$



We can then think about the "expected" benefit from contacting an individual that we predict will open an account. (Because we are only contacting people we predict will open the account, there are only two outcomes we care about: true positives (TP, people we predict will open and do open) and false positives (FP, people we predict will open but do not). Let's suppose that the TP and FP from our prediction rules tell us what would happen if we contacted new people.

The expected benefit is then just

$Pr(TP)*(R-C) + Pr(FP)(-C)$

We can use our data and prediction rules to fill in estimates of $Pr(TP)$ and $Pr(FP)$ from the different models.

In the following, we are going to contact anyone for whom the predicted probability of take-up is greater than 50%.

**Are there other things we are abstracting from here that might be important?**


In [ ]:
# Calculate expected profit

# Confusion matrix for constant model
tn, fp, fn, tp = confusion_matrix(y_val, y_pred_constant).ravel()
# Confusion matrix for logistic model
tn_log, fp_log, fn_log, tp_log = confusion_matrix(y_val, y_pred_logistic).ravel()
# Confusion matrix for random forest model
tn_rf, fp_rf, fn_rf, tp_rf = confusion_matrix(y_val, y_pred_rf).ravel()
# Confusion matrix for boosting model
tn_gbc, fp_gbc, fn_gbc, tp_gbc = confusion_matrix(y_val, y_pred_gbc).ravel()
# Confusion matrix for boosting model (early stopping)
tn_gbcES, fp_gbcES, fn_gbcES, tp_gbcES = confusion_matrix(y_val, y_pred_gbcES).ravel()

# Hypothetical value of customer opening deposit
R = 900
# Hypothetical "benefit" of contacting customer
C = -100

# Expected profit from constant model
N = tn+fp+fn+tp  # Total number of observations
Epi_con = (tp/N)*(R+C)+(fp/N)*C
# Expected profit from logistic model
Epi_log = (tp_log/N)*(R+C)+(fp_log/N)*C
# Expected profit from random forest model
Epi_rf = (tp_rf/N)*(R+C)+(fp_rf/N)*C
# Expected profit from boosting model
Epi_gbc = (tp_gbc/N)*(R+C)+(fp_gbc/N)*C
# Expected profit from boosting model (early stopping)
Epi_gbcES = (tp_gbcES/N)*(R+C)+(fp_gbcES/N)*C


print(f"Expected profit from constant model: {Epi_con}")
print(f"Expected profit from logistic model: {Epi_log}")
print(f"Expected profit from random forest model: {Epi_rf}")
print(f"Expected profit from boosted tree model: {Epi_gbc}")
print(f"Expected profit from boosted tree model (early stopping): {Epi_gbcES}")


An alternate way to think about evaluating the prediction rules is to consider choosing the threshold for contacting based on the following exercise:



1.   First, order all individuals in the sample according to predicted probability of take-up.
2.   Consider varying the threshold for contact (or alternatively the fraction of the sample to contact) between 1 (contact no one) and 0 (contact everyone)
3.   Choose the model and threshold that gives the highest "profit." (All the true ones give a benefit of R-C, and all the true zeros give a benefit of -C.)



In [ ]:
# Creating a DataFrame with the true values and predicted probabilities
datalog = pd.DataFrame({'true': y_val, 'prob': y_pred_prob_logistic})
datalog.sort_values(by='prob', ascending=False, inplace=True)

datarf = pd.DataFrame({'true': y_val, 'prob': y_pred_prob_rf})
datarf.sort_values(by='prob', ascending=False, inplace=True)

datagbc = pd.DataFrame({'true': y_val, 'prob': y_pred_prob_gbc})
datagbc.sort_values(by='prob', ascending=False, inplace=True)

datagbcES = pd.DataFrame({'true': y_val, 'prob': y_pred_prob_gbcES})
datagbcES.sort_values(by='prob', ascending=False, inplace=True)

# Calculate "profit" from contacting people ordered by their predicted probability
datalog['profit'] = np.cumsum(datalog['true']*(R-C)+(1-datalog['true'])*C)
datalog['cumulative_percentage'] = np.arange(1, len(datalog) + 1) / len(datalog)

datarf['profit'] = np.cumsum(datarf['true']*(R-C)+(1-datarf['true'])*C)
datarf['cumulative_percentage'] = np.arange(1, len(datarf) + 1) / len(datarf)

datagbc['profit'] = np.cumsum(datagbc['true']*(R-C)+(1-datagbc['true'])*C)
datagbc['cumulative_percentage'] = np.arange(1, len(datagbc) + 1) / len(datagbc)

datagbcES['profit'] = np.cumsum(datagbcES['true']*(R-C)+(1-datagbcES['true'])*C)
datagbcES['cumulative_percentage'] = np.arange(1, len(datagbcES) + 1) / len(datagbcES)

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(datalog['cumulative_percentage'], datalog['profit'], label='Profit - Logistic')
plt.plot(datarf['cumulative_percentage'], datarf['profit'],
         label='Profit - Random Forest')
plt.plot(datagbc['cumulative_percentage'], datagbc['profit'],
         label='Profit - Boosted Tree')
plt.plot(datagbcES['cumulative_percentage'], datagbcES['profit'],
         label='Profit - Boosted Tree (early stopping)')
plt.xlabel('Percentage of samples')
plt.ylabel('Profit')
plt.title('Profit Chart')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

In [ ]:
# Get predicted probability in row corresponding to maximum profit (random forest)
rfprob = datarf.loc[datarf['profit'].idxmax(),'prob']
print(f"Predicted takeup probability threshold (random forest): {rfprob}")
rfprof = datarf.loc[datarf['profit'].idxmax(),'profit']
print(f"Maximum profit (random forest): {rfprof}")

# Let's calculate accuracy using this decision threshold
rfpredictions = (y_pred_prob_rf >= rfprob).astype(int)
rfaccuracy = np.mean(rfpredictions == y_val)
print(f"Accuracy (random forest): {rfaccuracy}")

# Get predicted probability in row corresponding to maximum profit (boosted trees)
gbcprob = datarf.loc[datagbc['profit'].idxmax(),'prob']
print(f"Predicted takeup probability threshold (boosted trees): {gbcprob}")
gbcprof = datarf.loc[datagbc['profit'].idxmax(),'profit']
print(f"Maximum profit (boosted trees): {gbcprof}")

# Let's calculate accuracy using this decision threshold
gbcpredictions = (y_pred_prob_gbc >= gbcprob).astype(int)
gbcaccuracy = np.mean(gbcpredictions == y_val)
print(f"Accuracy (boosted trees): {gbcaccuracy}")

# Get predicted probability in row corresponding to maximum profit (boosted trees, early)
gbcESprob = datarf.loc[datagbcES['profit'].idxmax(),'prob']
print(f"Predicted takeup probability threshold (boosted trees, early): {gbcESprob}")
gbcESprof = datarf.loc[datagbcES['profit'].idxmax(),'profit']
print(f"Maximum profit (boosted trees, early): {gbcESprof}")

# Let's calculate accuracy using this decision threshold
gbcESpredictions = (y_pred_prob_gbcES >= gbcESprob).astype(int)
gbcESaccuracy = np.mean(gbcESpredictions == y_val)
print(f"Accuracy (boosted trees, early): {gbcESaccuracy}")


# Interpretation (using the random forest model)

There are many variable importance measures and other interpretation devices used with "black box" learners. Here we're just going to look at a few of them.

A variable importance measure is computed by default when you estimate a tree-based model in `sklearn`. This importance measure is computed by looking at the variable used at each split in the tree and the improvement in fit ("decrease in *impurity*") from making the split. This type of measure is computed from the training data and is thus an "in-sample" measure of importance.

The variable importances are then normalized to sum to 1.

In [ ]:
rf_importances = pd.Series(rf_model.feature_importances_, index=X_train.columns)
rf_importances.sort_values(ascending=True, inplace=True)

plt.figure(figsize=(10, 14))
rf_importances.plot.barh(color='green')
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.title("Random Forest Feature Importance - Built-in Method")
plt.show()

In [ ]:
top_10_rf_importances = rf_importances.tail(10)
top_10_rf_importances.plot.barh(color='green')
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.title("Random Forest Feature Importance (Top 10)- Built-in Method")
plt.show()

In [ ]:
PartialDependenceDisplay.from_estimator(rf_model, X_val, [X_val.columns.get_loc('euribor3m')], kind='both')
plt.show()

In [ ]:
col_names = ['euribor3m', 'nr.employed', 'never_contacted_1']  # List of column names to search for
col_indices = [X_val.columns.get_loc(col) for col in col_names]

fig, ax = plt.subplots(figsize=(10, 6))
PartialDependenceDisplay.from_estimator(rf_model, X_val, col_indices,
                                        categorical_features=[col_indices[2]],
                                        ax = ax)
plt.show()

In [ ]:
single_X = X_val.sample(n=1, random_state=42)

rfExplainer = shap.TreeExplainer(rf_model)
shap_values = rfExplainer.shap_values(single_X)

In [ ]:
print(f"Expected value: {rfExplainer.expected_value[1]}")
print(f"Random forest forecast: {rf_model.predict_proba(single_X)[0,1]}")
print(f"Sum of SHAP values: {sum(shap_values[0,:,1])}")
print(f"Expected value + sum of SHAP: {rfExplainer.expected_value[1] + sum(shap_values[0,:,1])}")

In [ ]:
shap.plots.force(rfExplainer.expected_value[1], shap_values[0,:,1], single_X.iloc[0], matplotlib=True)
plt.show()

In [ ]:
small_X_val = X_val.sample(n=100, random_state=42)

rfExplainer = shap.TreeExplainer(rf_model)
shap_values = rfExplainer.shap_values(small_X_val)

In [ ]:
shap.summary_plot(shap_values[:,:,1], small_X_val, plot_type="bar")

In [ ]:
shap.summary_plot(shap_values[:,:,1], small_X_val)

In [ ]:
def drop_column_importance(baseline_score, X_train, y_train, X_test, y_test, metric_fn, use_proba=False):
    """
    Calculate feature importance by dropping each column and measuring the performance drop.

    Parameters:
    - baseline_score: The performance score of the model with all features.
    - X_train: Training feature dataset.
    - y_train: Training target dataset.
    - X_test: Testing feature dataset.
    - y_test: Testing target dataset.
    - metric_fn: A function to evaluate the performance metric (e.g., accuracy_score, f1_score, roc_auc_score).
    - use_proba: Boolean indicating whether the metric function requires predicted probabilities (e.g., for roc_auc_score).

    Returns:
    - A sorted list of tuples (feature_name, importance) where importance is the percentage drop in performance.
    """
    importances = []

    for col in X_train.columns:
        # Drop the column
        X_train_dropped = X_train.drop(columns=[col])
        X_test_dropped = X_test.drop(columns=[col])

        # Get all the dummies
        X_train_dropped = pd.get_dummies(X_train_dropped, drop_first=False)
        X_test_dropped = pd.get_dummies(X_test_dropped, drop_first=False)

        # Align validation data to ensure same columns as training data
        X_test_dropped = X_test_dropped.reindex(
            columns=X_train_dropped.columns, fill_value=0)

        # Retrain the model without this feature using the same tuning parameters
        # as in the full data
        model_dropped = RandomForestClassifier(n_estimators=1000, min_samples_leaf=15, random_state=94)
        model_dropped.fit(X_train_dropped, y_train)

        # Re-evaluate the model
        if use_proba:
            predictions = model_dropped.predict_proba(X_test_dropped)[:, 1]
        else:
            predictions = model_dropped.predict(X_test_dropped)

        dropped_score = metric_fn(y_test, predictions)

        # Importance is the drop in performance
        importance = (baseline_score - dropped_score) / baseline_score
        importances.append((col, importance))

    # Return sorted feature importances
    importances.sort(key=lambda x: x[1], reverse=True)
    return importances


In [ ]:
# This block takes ~10 minutes to run

# Make sure we have the same data we used when we fit the original random forest
X_train = train.drop(columns = ['y'])
y_train = train['y']
X_val = val.drop(columns = ['y'])
y_val = val['y']

# Get dummy variables for our categorical features.
# We are just going to keep all the dummies
X_train = pd.get_dummies(X_train, drop_first = False)
X_val = pd.get_dummies(X_val, drop_first = False)

# Align validation data to ensure same columns as training data
X_val = X_val.reindex(columns=X_train.columns, fill_value=0)

# Evaluate the model performance on the test set according to favorite criterion
# Here, we're just using accuracy.
# We've already computed this, but we'll do it again.
baseline_score = accuracy_score(y_val, rf_model.predict(X_val))
print(f'Baseline accuracy: {baseline_score:.4f}')

# Get back data data without dummies created
X_train = train.drop(columns = ['y'])
y_train = train['y']
X_val = val.drop(columns = ['y'])
y_val = val['y']

# Calculate drop column feature importance
drop_col_importances = drop_column_importance(baseline_score,
                                              train.drop(columns = ['y']),
                                              train['y'],
                                              val.drop(columns = ['y']),
                                              val['y'], accuracy_score)

# Display the results
importance_df = pd.DataFrame(drop_col_importances, columns=['Feature', 'Importance'])
importance_df = importance_df.sort_values(by='Importance', ascending=False)

print(importance_df)

# Plot the feature importances
importance_df.plot(kind='bar', x='Feature', y='Importance',
                   title='Drop Importance - Accuracy', legend=False)
plt.show()


In [ ]:
# This block takes ~10 minutes to run
# Let's see how things look if we focus on AUC

# Make sure we have the same data we used when we fit the original random forest
X_train = train.drop(columns = ['y'])
y_train = train['y']
X_val = val.drop(columns = ['y'])
y_val = val['y']

# Get dummy variables for our categorical features.
# We are just going to keep all the dummies
X_train = pd.get_dummies(X_train, drop_first = False)
X_val = pd.get_dummies(X_val, drop_first = False)

# Align validation data to ensure same columns as training data
X_val = X_val.reindex(columns=X_train.columns, fill_value=0)

# Evaluate the model performance on the test set according to favorite criterion
# Here, we're using AUC
# We've already computed this, but we'll do it again.
baseline_score = roc_auc_score(y_val, rf_model.predict_proba(X_val)[:, 1])
print(f'Baseline accuracy: {baseline_score:.4f}')

# Get back data data without dummies created
X_train = train.drop(columns = ['y'])
y_train = train['y']
X_val = val.drop(columns = ['y'])
y_val = val['y']

# Calculate drop column feature importance
drop_col_importances = drop_column_importance(baseline_score,
                                              train.drop(columns = ['y']),
                                              train['y'],
                                              val.drop(columns = ['y']),
                                              val['y'], roc_auc_score,
                                              use_proba = True)

# Display the results
importance_df = pd.DataFrame(drop_col_importances, columns=['Feature', 'Importance'])
importance_df = importance_df.sort_values(by='Importance', ascending=False)

print(importance_df)

# Plot the feature importances
importance_df.plot(kind='bar', x='Feature', y='Importance',
                   title='Drop Importance - AUC', legend=False)
plt.show()